# Content-based Filtering Model Inference

In [15]:
import csv
import pandas as pd
import tensorflow as tf

In [16]:
brand_choice_list = ['Oppo', 'Asus', 'Infinix', 'Samsung', 'Vivo', 'Huawei', 'Apple', 'Realme', 'Xiaomi', 'Poco', 'lainnya/tidak ada']

In [17]:
user_model = tf.keras.models.load_model('user_model.keras')

Preprocess data into input compatible with the model.

In [18]:
def preprocess_user(x):
  one_hot = x[-1]
  one_hot = tf.one_hot(brand_choice_list.index(one_hot), len(brand_choice_list))
  x = tf.cast(tf.convert_to_tensor(x[:-1]), tf.float32)
  return tf.expand_dims(tf.concat([x, one_hot], 0), 0)

In [19]:
a = [5,5,5,5,5,5,'Apple'] # performance, camera, battery, software, RAM, storage, favorite brand
preprocessed_output = preprocess_user(a)

In [20]:
u_vector = user_model.predict(preprocessed_output)

1/1 [==============================] - 0s 63ms/step


In [27]:
p_vector = pd.read_csv('phone_vector.csv', header=None)

In [39]:
p_vector = tf.convert_to_tensor(p_vector)
p_vector = tf.cast(p_vector, tf.float32)

Predict phone ratings for a user.

In [45]:
rating_pred_list = []
for p in p_vector:
  rating_pred_list.append(tf.tensordot(u_vector, tf.expand_dims(p, 0), axes=(1,1)) * 5)

Get phone name.

In [50]:
phone_pred_list = pd.read_csv('phone_dataset_cleaned.csv')

In [51]:
phone_pred_list = phone_pred_list['name'].tolist()

Sort ratings to get top 10 recommended phones for a user.

In [52]:
top_10_phone = list(zip(rating_pred_list, phone_pred_list))
top_10_phone.sort(reverse=True)
rating_pred_list, phone_pred_list = zip(*top_10_phone)
rating_pred_list, phone_pred_list = list(rating_pred_list), list(phone_pred_list)

In [53]:
top_10_list = []
for i in range(10):
  top_10_list.append(phone_pred_list[i])
  print(i+1, phone_pred_list[i], rating_pred_list[i].numpy())

1 iPhone 15 Pro [[4.6715217]]
2 iPhone 14 Pro [[4.6715097]]
3 iPhone 15 Pro Max [[4.6714597]]
4 iPhone 14 Pro Max [[4.6714478]]
5 iPhone 13 [[4.671291]]
6 iPhone 14 Plus [[4.66598]]
7 Samsung Galaxy Z Fold5 [[4.6275544]]
8 Samsung Galaxy S24 [[4.627504]]
9 Samsung Galaxy S24 Ultra [[4.6262345]]
10 Samsung Galaxy S24+ [[4.6193595]]


In [54]:
top_10_list

['iPhone 15 Pro',
 'iPhone 14 Pro',
 'iPhone 15 Pro Max',
 'iPhone 14 Pro Max',
 'iPhone 13',
 'iPhone 14 Plus',
 'Samsung Galaxy Z Fold5',
 'Samsung Galaxy S24',
 'Samsung Galaxy S24 Ultra',
 'Samsung Galaxy S24+']